In [12]:
from fredapi import Fred
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
idx = pd.IndexSlice
today = dt.datetime.today().strftime('%Y_%m_%d')
today

'2022_02_14'

# OECD
## 1.1 Unemployment

In [5]:
#os.listdir('Input/')

In [4]:
oe_up = pd.read_excel('Input/OECD/OECD_UI.xlsx', header=5)
oe_up = oe_up.set_index('Time')
oe_up = oe_up.iloc[1:,2:].transpose()
oe_up.index = pd.to_datetime(oe_up.index)
oe_up = oe_up.replace('..',np.nan).astype(float)
oe_up.head(3)

Time,Australia,Austria,Belgium,Canada,Chile,Colombia,Costa Rica,Czech Republic,Denmark,Estonia,...,Slovenia,Spain,Sweden,Turkey,United Kingdom,United States,Data extracted on 31 Jan 2022 19:43 UTC (GMT) from OECD.Stat,Legend:,B:,P:
1999-01-01,7.119821,4.3,8.9,7.9,8.378542,NaN,NaN,8.0,5.0,10.1,...,7.3,14.8,7.3,NaN,6.227229,4.3,NaN,NaN,NaN,NaN
1999-02-01,7.115868,4.2,9.0,7.9,8.821516,NaN,NaN,8.2,5.0,10.1,...,7.2,14.3,7.0,NaN,6.176034,4.4,NaN,NaN,NaN,NaN
1999-03-01,7.013355,4.2,9.0,7.9,9.316011,NaN,NaN,8.4,5.0,10.1,...,7.2,13.9,7.0,NaN,6.154285,4.2,NaN,NaN,NaN,NaN


## 1.2 GDP

- The OECD's quarterly national accounts (QNA)
- Gross domestic product- expenditure approach
    - VPVOBARSA: US dollars, volume estimates, fixed PPPs, OECD reference year, annual levels, seasonally adjusted 

### 1.2.A Import data

In [6]:
path = 'Input/OECD/'
os.listdir(path)

['OECD_GDP.xlsx', 'OECD_UI.xlsx', 'OECD_GDP_Real.xlsx']

In [10]:
oe_gdp = pd.read_excel(path + 'OECD_GDP_Real.xlsx', header=5)
oe_gdp = oe_gdp.set_index('Period')
oe_gdp = oe_gdp.iloc[1:,3:].transpose()
oe_gdp = oe_gdp.iloc[:, :45]
oe_gdp.index = pd.PeriodIndex(pd.to_datetime([x[1] + '-' +x[0] for x in oe_gdp.index.str.split('-', expand=True)]), freq='Q')
oe_gdp = oe_gdp.replace('..',np.nan).astype(float)
oe_gdp.head(2)

Period,Australia,Austria,Belgium,Canada,Chile,Colombia,Costa Rica,Czech Republic,Denmark,Estonia,...,Turkey,United Kingdom,United States,Euro area (19 countries),European Union – 27 countries (from 01/02/2020),G7,NAFTA,OECD - Europe,G20,OECD - Total
2007Q1,897248.4,405326.3,478332.1,1405089.7,308480.7,445083.5,62847.4,322569.7,270244.4,38965.9,...,1367327.9,2563323.1,16205028.9,13557381.2,16298179.3,33845444.1,19505011.0,20334930.8,68918126.1,48446142.1
2007Q2,902701.7,410115.8,480517.1,1418703.7,311361.9,449708.4,64465.9,325307.7,269034.0,39117.9,...,1367745.4,2575210.4,16308487.4,13654825.8,16413620.9,34030741.8,19636670.6,20469648.3,69784907.0,48761393.4


### 1.2.B Compare with FRED Data 

In [11]:
oe_gdp.loc[:, 'United States'].pct_change(1).apply(lambda r: (1+r)**4-1).multiply(100).tail(4)

2021Q1    6.278175
2021Q2    6.726284
2021Q3    2.303425
2021Q4    6.887441
Freq: Q-DEC, Name: United States, dtype: float64

In [13]:
api_file = open('api_key.txt')
api_code = "".join(api_file.readlines())
fred = Fred(api_key=api_code)
df_gdp = fred.get_series('GDPC1')
df_gdp.pct_change(1).apply(lambda r: (1+r)**4-1).multiply(100).tail(4)

2021-01-01    6.278177
2021-04-01    6.726283
2021-07-01    2.303426
2021-10-01    6.887441
dtype: float64

### 1.3 Normalize

In [14]:
norm = oe_gdp['2020Q1':'2020Q1']
oe_gdp_norm = oe_gdp.transform(lambda x: x/x[-8]).multiply(100).loc['2020':,:]
oe_gdp_norm.tail(3)

Period,Australia,Austria,Belgium,Canada,Chile,Colombia,Costa Rica,Czech Republic,Denmark,Estonia,...,Turkey,United Kingdom,United States,Euro area (19 countries),European Union – 27 countries (from 01/02/2020),G7,NAFTA,OECD - Europe,G20,OECD - Total
2021Q2,102.066866,99.918928,101.679519,99.467412,103.060037,99.525968,100.788681,98.871784,101.897315,105.707274,...,109.282505,100.069611,102.196698,101.027754,100.978162,101.200600,101.616860,101.623062,103.998646,101.427227
2021Q3,100.105061,103.729118,103.753442,100.795419,108.081082,105.207847,104.688314,100.420025,102.969790,106.401324,...,112.243434,101.131578,102.780188,103.315931,103.162840,102.110676,102.148712,103.784516,NaN,102.538801
2021Q4,NaN,101.424939,104.277196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,104.505965,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
date = '2021Q3'
oe_gdp_norm[date:date].transpose().sort_values(date, ascending=False)

,2021Q3
Period,
Ireland,117.927512
Turkey,112.243434
Chile,108.081082
Luxembourg,106.558558
Estonia,106.401324
France,106.252217
Slovenia,106.166099
Colombia,105.207847
Israel,104.985472
